In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [10]:
raw_data = pd.read_csv(r'data_film.txt', sep= '\t', names= ['titre','date','duree','type','note','nombre avis','avis']) 

In [11]:
raw_data

,titre,date,duree,type,note,nombre avis,avis
0,Black Panther: Wakanda Forever,9 novembre 2022,2h 42min,"['Action', 'Aventure', 'Fantastique']",3.4,512,['\nComme le monde entier a été choqué d\'appr...
1,Mascarade,1 novembre 2022,2h 14min,['Comédie dramatique'],3.6,575,"[""\nMoins original que la Belle Epoque, Mascar..."
2,Les Amandiers,16 novembre 2022,2h 05min,['Comédie dramatique'],3.7,115,['\nOn a bien compris toute l\'implication de ...
3,Couleurs de l'incendie,9 novembre 2022,2h 16min,"['Historique', 'Drame']",4.0,368,['\nLa mort de son père laisse Madeline Perico...
4,Novembre,5 octobre 2022,1h 47min,"['Thriller', 'Policier']",4.1,661,"['\nAprès ""Bac Nord"", qui avait divisé la crit..."
5,Le Menu,23 novembre 2022,1h 48min,['Thriller'],3.5,125,['\nHors d\'oeuvre:Un intrigant et mystérieux ...
6,"Simone, le voyage du siècle",12 octobre 2022,2h 20min,['Biopic'],4.2,660,"['\nFilm évènement de l’automne, dont la sorti..."
7,Reste un peu,16 novembre 2022,1h 33min,['Comédie'],3.5,289,"[""\nPeut-on apprécier Reste un peu si l'on n'a..."
8,Les Miens,23 novembre 2022,1h 25min,['Drame'],3.7,91,['\nLes Miens est le film le plus personnel ré...
9,La Conspiration du Caire,26 octobre 2022,1h 59min,['Thriller'],4.0,210,"[""\nAvec Boy from Heaven, qui aurait pu s'inti..."
